In [4]:
from selenium import webdriver
from datetime import date, datetime, timedelta
import time
import os
import pandas as pd
from outlook_send import send_email

from IPython.display import display, HTML

# constant to establish download folder path, only need to change this to change location
DOWNLOAD_FOLDER = r"C:\Users\boa.sokchu\Downloads"

# list of files before downloading
before = os.listdir(DOWNLOAD_FOLDER)

# activate chrome driver
browser = webdriver.Chrome()
browser.maximize_window()
browser.get("https://boa.3plsystemscloud.com/")

# page elements to login
boa_user = browser.find_element_by_id("txb-username")
boa_pw = browser.find_element_by_id("txb-password")
login_button = browser.find_element_by_id("ctl00_ContentBody_butLogin")

# login credentials
boa_user.send_keys("sokchu@boalogistics.com")
boa_pw.send_keys("Immuneact10!")
login_button.click()

# enter report code into report_code variable
# "New_Carrier" report
report_code = "35D341379474"
url = "https://boa.3plsystemscloud.com/App_BW/staff/Reports/ReportViewer.aspx?code="+report_code
browser.get(url)

# sets up start date and end date for filter
# checks today's date to calculate next 2 business days

today = date.today()
weekday = datetime.weekday(today)
lastweek = 7
yesterday = 1

s_date = today - timedelta(lastweek)
e_date = today - timedelta(yesterday)

start = s_date.strftime("%m/%d/%Y 00:00:00")
end = e_date.strftime("%m/%d/%Y 23:59:59")

# for manually pulling specific weeks

#start = s_date.strftime("05/24/2021 00:00:00")
#end = e_date.strftime("05/30/2021 23:59:59")

# set up variables for parameter fields
startbox = browser.find_element_by_xpath("//td[4]/input[@class='filter between'][1]")
endbox = browser.find_element_by_xpath("//td[4]/input[@class='filter between'][2]")


# inserts new parameters
startbox.clear()
startbox.send_keys(start)
endbox.clear()
endbox.send_keys(end)

# save & view report, then download
save_button = browser.find_element_by_id("ctl00_ContentBody_butSaveView").click()
browser.implicitly_wait(3)
download = browser.find_element_by_id("ctl00_ContentBody_butExportToExcel").click()

#need to wait a few seconds before continuing to allow for file to finish downloading.
time.sleep(3)

browser.quit()

#compares list of files in Downloads folder after downloading file to extract filename
after = os.listdir(DOWNLOAD_FOLDER)
change = set(after) - set(before)

if len(change) == 1:
    file_name = change.pop()
    print(file_name + " downloaded.")
else:
    print ("More than one file or no file downloaded")
    
# sets filepath to downloaded file and create DataFrame from file 
# *output file extension is .xls but is actually.html format

filepath = DOWNLOAD_FOLDER + "\\" + file_name
data = pd.read_html(filepath)
df = data[0]

# list of substring to look for

sub = '_'
sub2 = 'YP_'
sub3 = 'YV_'
sub4 = 'DS_'

carrier_name = df[df['Carrier Name'].str.startswith(sub2, na=False) | df['Carrier Name'].str.startswith(sub3, na=False) | df['Carrier Name'].str.startswith(sub4, na=False)]

#print(carrier_name)

df = carrier_name

#saves as .xlsx file

output_path = DOWNLOAD_FOLDER + "\\" + "REVISED" + file_name + ".xlsx"

writer = pd.ExcelWriter(output_path, engine="xlsxwriter")
df.to_excel(writer, sheet_name='DAILYREPORT')
writer.save()

print("Saved file to " + output_path + "!")

# Pass arguments below in following order: To Address, Subject, Email Body, Path to file to attach
send_email('Jenny@boalogistics.com, data@boalogistics.com ',
           'New Carriers ' + start[0:10] + ' - ' + end[0:10],
           'Good Morning Jennifer,\n\nPlease see attached report for new carriers added by Carrier Sales last week.\n\nThank you,\n\nSokchu Hwang',output_path)

NewCarrierReport20210823.xls downloaded.
Saved file to C:\Users\boa.sokchu\Downloads\REVISEDNewCarrierReport20210823.xls.xlsx!
email sent


True

In [5]:
today.strftime("%m/%d/%Y")

'08/13/2021'